In [6]:
import pandas as pd
import numpy as np

# 讀取 CSV 檔案
data = pd.read_csv('aqi_breakpoints.csv')

In [7]:
# 顯示資料框的前幾行
data.head()

,siteid,aqi,pollutant,status,so2,o3,o3_8hr,no2,windspeed,winddirec,...,longitude,latitude,year,so2_aqi,o3_aqi,o3_8hr_aqi,pm2.5_avg_aqi,pm10_avg_aqi,co_8hr_aqi,no2_aqi
0,25.0,49.0,NaN,良好,0.5,9.7,26.0,17.4,0.3,291.0,...,120.898693,24.696907,2022,1,9,21,48,22,3,28
1,25.0,72.0,細懸浮微粒,普通,0.7,9.8,23.0,19.8,0.3,189.0,...,120.898693,24.696907,2022,2,9,18,72,31,3,32
2,25.0,92.0,細懸浮微粒,普通,4.0,3.0,19.0,22.9,0.9,201.0,...,120.898693,24.696907,2022,10,3,15,91,40,3,37
3,25.0,91.0,細懸浮微粒,普通,2.7,12.1,16.0,12.5,0.7,122.0,...,120.898693,24.696907,2022,6,11,13,91,40,3,20
4,25.0,90.0,細懸浮微粒,普通,2.0,12.7,14.0,9.0,0.7,262.0,...,120.898693,24.696907,2022,5,12,11,89,40,3,15


In [8]:
# 檢查指定欄位的 NaN 值
columns_to_check = ['so2', 'o3', 'o3_8hr', 'no2', 'windspeed', 'winddirec', 'co_8hr', 'pm2.5_avg', 'pm10_avg']
# 檢查指定的 columns 是否包含 NaN 值
nan_check = data[columns_to_check].isna().sum()

# 顯示結果
print(nan_check)

so2          0
o3           0
o3_8hr       0
no2          0
windspeed    0
winddirec    0
co_8hr       0
pm2.5_avg    0
pm10_avg     0
dtype: int64


In [9]:
# 使用groupby和agg來創建站點經緯度的字典
site_coordinates_df = data.groupby('siteid').agg({'longitude': 'first', 'latitude': 'first'}).reset_index()
siteid_to_coordinates = site_coordinates_df.set_index('siteid').T.to_dict('dict')

# 顯示結果
site_coordinates_df

,siteid,longitude,latitude
0,25.0,120.898693,24.696907
1,26.0,120.820115,24.564992
2,27.0,120.759568,24.382484
3,28.0,120.742524,24.256997
4,29.0,120.568794,24.225628
5,30.0,120.678444,24.099611
6,31.0,120.641092,24.151958
7,32.0,120.616917,24.162197
8,33.0,120.541519,24.066000
9,34.0,120.469061,24.131672


In [11]:
import pandas as pd

# 讀取 CSV 檔案
data = pd.read_csv('aqi_breakpoints.csv')

# 使用groupby和agg來創建站點經緯度的字典
site_coordinates_df = data.groupby('siteid').agg({'longitude': 'first', 'latitude': 'first'}).reset_index()
siteid_to_coordinates = site_coordinates_df.set_index('siteid').T.to_dict('dict')

# 獲取經緯度的最大值和最小值
longitudes = [coord['longitude'] for coord in siteid_to_coordinates.values()]
latitudes = [coord['latitude'] for coord in siteid_to_coordinates.values()]
min_long, max_long = min(longitudes), max(longitudes)
min_lat, max_lat = min(latitudes), max(latitudes)

matrix_size = 5

# 創建 4x4 矩陣
matrix = [[[] for _ in range(matrix_size)] for _ in range(matrix_size)]

# 將經緯度映射到 4x4 矩陣的索引範圍
def map_to_index(value, min_value, max_value, size):
    return int((value - min_value) / (max_value - min_value) * (size - 1))

# 將站點放置在矩陣中並記錄位置
site_positions = {}

# 將站點放置在矩陣中
for siteid, coordinates in siteid_to_coordinates.items():
    long_idx = map_to_index(coordinates['longitude'], min_long, max_long, matrix_size)
    lat_idx = map_to_index(coordinates['latitude'], min_lat, max_lat, matrix_size)
    
    # 如果該位置已經有站點，尋找附近的空位置放置新站點
    if matrix[lat_idx][long_idx]:
        found_empty = False
        search_range = 1
        while not found_empty and search_range < matrix_size:
            for i in range(max(0, lat_idx - search_range), min(matrix_size, lat_idx + search_range + 1)):
                for j in range(max(0, long_idx - search_range), min(matrix_size, long_idx + search_range + 1)):
                    if not matrix[i][j]:
                        matrix[i][j] = siteid
                        site_positions[siteid] = (i, j)
                        found_empty = True
                        break
                if found_empty:
                    break
            search_range += 1
        if not found_empty:
            print(f"Warning: Could not place siteid {siteid} within the matrix.")
    else:
        matrix[lat_idx][long_idx] = siteid
        site_positions[siteid] = (lat_idx, long_idx)

# 打印矩陣
for row in matrix:
    print(row)


[33.0, 31.0, 32.0, 35.0, 69.0]
[34.0, 28.0, 30.0, 36.0, 72.0]
[37.0, 29.0, 27.0, 38.0, []]
[41.0, 83.0, [], 26.0, []]
[[], [], [], 25.0, []]


In [13]:
site_positions

{25.0: (4, 3),
 26.0: (3, 3),
 27.0: (2, 2),
 28.0: (1, 1),
 29.0: (2, 1),
 30.0: (1, 2),
 31.0: (0, 1),
 32.0: (0, 2),
 33.0: (0, 0),
 34.0: (1, 0),
 35.0: (0, 3),
 36.0: (1, 3),
 37.0: (2, 0),
 38.0: (2, 3),
 41.0: (3, 0),
 69.0: (0, 4),
 72.0: (1, 4),
 83.0: (3, 1)}